# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

In [20]:
import json
import pandas as pd
import os
import time
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts, chain

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(
        columns=["__row_index"]
    )
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

,dt,tx_ct
0,2023-08-28T00:00:00.000Z,356396
1,2023-08-29T00:00:00.000Z,1019692
2,2023-08-30T00:00:00.000Z,1027316
3,2023-08-31T00:00:00.000Z,1010128
4,2023-09-01T00:00:00.000Z,1006635
5,2023-09-02T00:00:00.000Z,1024172
6,2023-09-03T00:00:00.000Z,1038128
7,2023-09-04T00:00:00.000Z,610216


### Dune

* API Docs: https://dune.com/docs/api/

In [6]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [7]:
dune_query_id = 2847682  # Query URL: https://dune.com/queries/2847682
results_df = query_dune(dune_query_id)
results_df

KeyError: 'result'

### Weather API

* API Docs: https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/

In [21]:
weather_url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London%2CUK/2022-07-15/2022-07-20?unitGroup=metric&contentType=csv&include=days&key={WEATHER_API_KEY}"

weather_response = requests.get(weather_url)

results_df = pd.read_csv(weather_url)

results_df

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,"London, England, United Kingdom",2022-07-15,25.5,14.6,20.6,25.5,14.6,20.6,8.3,46.5,...,17.8,9,NaN,2022-07-15T05:00:53,2022-07-15T21:11:28,0.56,Clear,Clear conditions throughout the day.,clear-day,"03769099999,03680099999,D5621,03781099999,0367..."
1,"London, England, United Kingdom",2022-07-16,27.8,15.3,21.8,27.1,15.3,21.7,10.9,51.8,...,17.5,8,NaN,2022-07-16T05:02:05,2022-07-16T21:10:27,0.59,Clear,Clear conditions throughout the day.,clear-day,"03769099999,03680099999,D5621,03781099999,0367..."
2,"London, England, United Kingdom",2022-07-17,30.0,16.1,23.6,28.7,16.1,23.2,9.1,42.2,...,17.0,8,NaN,2022-07-17T05:03:18,2022-07-17T21:09:23,0.62,Clear,Clear conditions throughout the day.,clear-day,"03680099999,D5621,03781099999,03672099999,0377..."
3,"London, England, United Kingdom",2022-07-18,36.6,18.0,27.7,34.6,18.0,26.8,9.9,37.0,...,16.1,8,NaN,2022-07-18T05:04:33,2022-07-18T21:08:16,0.66,Clear,Clear conditions throughout the day.,clear-day,"03680099999,D5621,03781099999,03672099999,0377..."
4,"London, England, United Kingdom",2022-07-19,39.8,21.7,30.3,37.9,21.7,29.4,11.6,36.0,...,16.4,8,NaN,2022-07-19T05:05:49,2022-07-19T21:07:07,0.69,Rain,Clear conditions throughout the day with late ...,rain,"03680099999,D5621,03781099999,03672099999,D412..."
5,"London, England, United Kingdom",2022-07-20,26.6,19.1,22.7,26.6,19.1,22.7,15.4,64.9,...,14.3,6,NaN,2022-07-20T05:07:06,2022-07-20T21:05:55,0.75,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,"03769099999,03680099999,D5621,03781099999,0367..."


## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [4]:
# Create File from Query results
results_df.to_csv("query_results.csv", index=False)

In [5]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(["git", "add", "."])
        subprocess.run(["git", "commit", "-m", commit_message])
        subprocess.run(["git", "push"])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)


# Replace 'Your commit message here' with your desired commit message
commit_message = "Your commit message here"

git_add_commit_push(commit_message)

[main f116129] Your commit message here
 2 files changed, 45 insertions(+), 48 deletions(-)
Git add, commit, and push successful!


To https://github.com/Demoniarc/ocean_data_nfts
   f4c50e5..f116129  main -> main


## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [6]:
# Connect to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [7]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv("PRIVATE_KEY")
wallet = accounts.add(private_key)
print("MATIC Balance:", wallet.balance() / 1e18)

MATIC Balance: 7.505173214082293


In [8]:
# Create Data NFT
name = "Data NFT (2)"

# Link to File
result = subprocess.run(
    ["git", "config", "--get", "user.name"], stdout=subprocess.PIPE, text=True
)
github_username = result.stdout.strip().replace(" ", "")
url = f"https://raw.githubusercontent.com/{github_username}/ocean_data_nfts/main/query_results.csv"

# Set Gas Fees
priority_fee = chain.priority_fee
max_fee = (chain.base_fee * 2) + priority_fee
gas_limit = int((priority_fee + max_fee) / 1e5)
print("priority_fee:", priority_fee, "max_fee:", max_fee, "gas_limit:", gas_limit)

# Configure Transaction Variables
tx_dict = {
    "from": wallet,
    "priority_fee": priority_fee,
    "max_fee": max_fee,
    "gas_limit": gas_limit,
}

# Publish Data NFT
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, tx_dict)

time.sleep(30)

# Set Price to Free
datatoken.create_dispenser(tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")
print(f"  Link: https://market.oceanprotocol.com/asset/{ddo.did}")

priority_fee: 30016684671 max_fee: 229985019705 gas_limit: 2600017
Transaction sent: 0x298cda714b24bc41730b631abc5765a4c366481922ebdc7a933ddc989c99157a
  Max fee: 229.985019705 gwei   Priority fee: 30.016684671 gwei   Gas limit: 2600017   Nonce: 4
  ERC721Factory.createNftWithErc20 confirmed   Block: 47139461   Gas used: 1595406 (61.36%)   Gas price: 134.442513418 gwei



INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x043b46b064afe6312136ea8517a7e34dd790d574bf901f06c81d1768b54c3f072ea6430e906726bd1083a7f00e16e9f2940c5382ba4186bfc6d7db37bb90ca1c0db7048f14405f27eb8139ab58c34889201d1d0563a3a1ddaa3bb2691682aa832d4633b739c45586d1ba72d2706639b465d788412ffd06776194bb2d79e94d2a484c4e54476b704b77bce65994bc158f13bec1c39a15fcf89befe3b73e8780581dc4a9f9e07c39721c4a988a5fc61e8e7522c58cc0becfe4b65aa36196d9d36bfb71908da0326121187ce7aa70a87334ebabaf1fc584f36853d7981dc4b5b6fee18951fc5ecfcc8b00446b1999512aa070e61665c3cbfae8323e74e209a68c6234b25f34e38c3624e8f3ae097bfebe25775e876ce2feb7eed342dc87ae0577916912bba4d6368f69e6180f3918d2ad6f22be956c2efe1c7e6ffdaf7399d20930eed417138803ee53637a, encryptedEndpoint https://v4.provider.polygon.oceanprotocol.com/api/services/encrypt?chainId=137
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04b6d5d3f411afd5b3e561d1b4afb3bee086c39

Transaction sent: 0x25a49e33ff328f7d6a1e3077cb38c85d309a3ccd32417b1ce55de93b93fa2406
  Max fee: 229.985019705 gwei   Priority fee: 30.016684671 gwei   Gas limit: 2600017   Nonce: 5
  ERC721Template.setMetaData confirmed   Block: 47139466   Gas used: 211907 (8.15%)   Gas price: 132.491004556 gwei

Transaction sent: 0x49056d30acdb466953d0d909a12caba89a66e1d2c2ea3640cd0345ddddcb4600
  Max fee: 229.985019705 gwei   Priority fee: 30.016684671 gwei   Gas limit: 2600017   Nonce: 6
  ERC20Template.createDispenser confirmed   Block: 47139499   Gas used: 283122 (10.89%)   Gas price: 125.877321218 gwei

Data NFT published:
  data_nft: symbol=Data NFT (2), address=0x499ad9dbB468cc7B7B74306F40905671a2859088
  datatoken: symbol=DT1, address=0xeC2128A7050289C71a736e16af9dA9f38Ff4D4C7
  did=did:op:f3eca17db312166fd8dc2f0190cc85a5b9a9e319dc124ff7f8ae010f515d7791
  Link: https://market.oceanprotocol.com/asset/did:op:f3eca17db312166fd8dc2f0190cc85a5b9a9e319dc124ff7f8ae010f515d7791
